In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

In [2]:
tf.__version__

'2.6.0'

In [3]:
pip install tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow_datasets as tfds

In [5]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'd4rl_adroit_door',
 'd4rl_adroit_hammer',
 'd4rl_adroit_pen',
 'd4rl_adroit_relocate',
 'd4rl_mujoco_ant',
 'd4

In [78]:
train_set_raw = tfds.load("imdb_reviews", split='train[70%:]', as_supervised=True)
test_set_raw = tfds.load("imdb_reviews", split='test', as_supervised=True)

In [79]:
train_set_raw.take(3)

<TakeDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [80]:

train_examples_batch,train_labels_batch = next(iter(train_set_raw.batch(10)))
train_examples_batch


<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"Nine out of ten might seem like a high mark to give for a straight to video sci-fi movie that's been vilified at the US box office and roundly criticized as the poorest movie of Kurt Russell's career.<br /><br />I have my reasons.<br /><br />Firstly when you read negative reviews of this film, they usually start with the wooden nature of Russell's interpretation of Todd, the eponymous Soldier. I'm going to start here too, with my surprising statement that this is possibly the finest piece of acting I've seen Russell pull off. Todd is an emotional cripple and suffering from intense PTSD - this movie being written before the phenomenon was as widely recognized as it is now.<br /><br />The portrayal is spot on. Todd is withdrawn, uncommunicative and a loner. He suffers from irrational anxiety - keyed to a fever pitch by training that teaches him to analyze every movement and interaction with another human being for signs of betrayal a

In [81]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 0, 0, 1, 0, 0, 1, 1, 0], dtype=int64)>

In [82]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

In [83]:
hub_layer(train_examples_batch[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 4.7923617 , -4.525794  ,  7.1313257 ,  0.21008398, -2.6384459 ,
        -5.4846916 , -2.050298  ,  3.532944  ,  4.2297544 , -2.9579313 ,
        -2.2863336 ,  1.6684309 , -2.6761084 , -0.3572238 , -6.342297  ,
         3.9719656 ,  8.341165  , -4.8317847 , -6.654334  , -2.5732622 ],
       [ 2.5741632 , -0.8085664 ,  2.2234964 , -1.6681693 , -0.12159869,
        -0.474802  , -0.8598956 ,  0.5045804 ,  1.9454796 , -2.8245873 ,
        -0.97832465,  0.39860973, -1.2733508 , -1.0127535 , -1.6692569 ,
         1.9692352 ,  0.4376294 , -0.91439754, -2.7024016 , -0.53923404]],
      dtype=float32)>

In [84]:
train_examples_batch[:2]

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"Nine out of ten might seem like a high mark to give for a straight to video sci-fi movie that's been vilified at the US box office and roundly criticized as the poorest movie of Kurt Russell's career.<br /><br />I have my reasons.<br /><br />Firstly when you read negative reviews of this film, they usually start with the wooden nature of Russell's interpretation of Todd, the eponymous Soldier. I'm going to start here too, with my surprising statement that this is possibly the finest piece of acting I've seen Russell pull off. Todd is an emotional cripple and suffering from intense PTSD - this movie being written before the phenomenon was as widely recognized as it is now.<br /><br />The portrayal is spot on. Todd is withdrawn, uncommunicative and a loner. He suffers from irrational anxiety - keyed to a fever pitch by training that teaches him to analyze every movement and interaction with another human being for signs of betrayal an

In [85]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_6 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [90]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [91]:
model.fit(train_set_raw.shuffle(10000).batch(512),
         epochs=20)

Epoch 1/20
15/15 [==============================] - 1s 36ms/step - loss: 0.1766 - accuracy: 0.9416
Epoch 2/20
15/15 [==============================] - 1s 36ms/step - loss: 0.1668 - accuracy: 0.9452
Epoch 3/20
15/15 [==============================] - 1s 35ms/step - loss: 0.1571 - accuracy: 0.9497
Epoch 4/20
15/15 [==============================] - 1s 35ms/step - loss: 0.1489 - accuracy: 0.9539
Epoch 5/20
15/15 [==============================] - 1s 36ms/step - loss: 0.1413 - accuracy: 0.9573
Epoch 6/20
15/15 [==============================] - 1s 36ms/step - loss: 0.1338 - accuracy: 0.9596
Epoch 7/20
15/15 [==============================] - 1s 37ms/step - loss: 0.1259 - accuracy: 0.9637
Epoch 8/20
15/15 [==============================] - 1s 36ms/step - loss: 0.1193 - accuracy: 0.9671
Epoch 9/20
15/15 [==============================] - 1s 36ms/step - loss: 0.1126 - accuracy: 0.9696
Epoch 10/20
15/15 [==============================] - 1s 36ms/step - loss: 0.1060 - accuracy: 0.9709
Epoch 11/

In [103]:
model.predict(["This movie is so amazing, I really liked it!",
              "Awful movie, such a huge disappointment",
              "Wonderful and exceptional, I am very impressed"])

array([[0.9991082 ],
       [0.02137616],
       [0.9985489 ]], dtype=float32)

In [105]:
model.predict(["Not big of a fan"])

array([[0.28377032]], dtype=float32)